# Julia-Notebook zur Veranschaulichung des IIR-Filterentwurfs mit Bilineartransformation

## Initialisierung

In [1]:
using Interact, Plots, DSP, LaTeXStrings

HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-16824150541433749932\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-2065778099169685367\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio-jupyter-notebook\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<p\n    id=\"webio-warning-16824150541433749932\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    For troubleshooting, please see <a href=\"https://juliagizmos.github.io/WebIO.jl/latest/providers/ijulia/\">\n    the WebIO/IJulia documentation</a>.\n    <!-- TODO: link to installation docs. -->\n</p>\n")

Nützliche Funktionen:

In [2]:
function circle(r)
    θ = LinRange(0, 2*π, 500)
    sin.(θ), cos.(θ)
end

function zplane(z, p)
    plot(circle(1), seriestype = [:shape,], c = :black, fillalpha = .2, aspect_ratio = 1,
        label=false, legend=:bottomright)
    scatter!(real(z), imag(z), c = :blue, label="Nullstellen")
    xlabel!(L"\mathrm{Re}{z}")
    ylabel!(L"\mathrm{Im}{z}")
    scatter!(real(p), imag(p), c = :red, label="Polstellen")
end

zplane (generic function with 1 method)

## IIR-Filterentwurf mit Bilineartransformation

Designmethoden:
* **Butterworth**-Filter: `Butterworth`
* **Tschebyscheff** mit Welligkeit im Durchlassbereich: `Chebyshev1`
* **Tschebyscheff** mit Welligkeit im Sperrbereich: `Chebyshev2`
* **Elliptisches** oder Cauer-Filter: `Elliptic`

Parameter:
* Filterordnung des analogen Musterfilters: $N$
* Normierte Grenzfrequenz: $f_{1}/f_s$
* Welligkeit im Durchlassbereich: $\Delta H_\text{pass}$
* Normierte Grenzfrequenz: $f_{2}/f_s$
* Welligkeit im Sperrbereich: $\Delta H_\text{stop}$

Frequenzgang:

In [38]:
@manipulate for N in 2:10, f1_norm in .05:.01:.2, ΔHpass in 0.01:.01:0.2, f2_norm in 0.25:.01:.45, ΔHstop in 0.01:.01:0.2, 
        responsetype_sel = Dict("TP" => 1, "HP" => 2, "BP" => 3, "BS" => 4),
        designmethod_sel = Dict("Butterworth" => 1, "Chebyshev1" => 2, "Chebyshev2" => 3, "Elliptic" => 4)
    ΔHpassdB = -20*log10(1-ΔHpass)
    ΔHstopdB = -20*log10(ΔHstop)
    if responsetype_sel == 1
        responsetype = Lowpass(f1_norm*2)
    elseif responsetype_sel == 2
        responsetype = Highpass(f1_norm*2)
    elseif responsetype_sel == 3
        responsetype = Bandpass(f1_norm*2, f2_norm*2)
    elseif responsetype_sel == 4
        responsetype = Bandstop(f1_norm*2, f2_norm*2)
    end
    if designmethod_sel == 1
        designmethod = Butterworth(N)
    elseif designmethod_sel == 2
        designmethod = Chebyshev1(N, ΔHpassdB)
    elseif designmethod_sel == 3
        designmethod = Chebyshev2(N, ΔHstopdB)
    elseif designmethod_sel == 4
        designmethod = Elliptic(N, ΔHpassdB, ΔHstopdB)
    end
    filter = digitalfilter(responsetype, designmethod)
    f_norm = 0:.001:.5
    H = freqz(filter, f_norm*2*π)
    plot(f_norm, abs.(H), label=false, lw=2, c = :blue)
    vline!([f1_norm], label = L"f = f_1", c = :green)
    if responsetype_sel == 3 || responsetype_sel == 4
        vline!([f2_norm], label = L"f = f_2", c = :green)
    end
    xlims!((0,.5))
    ylims!((0,1.2))
    xlabel!(L"f/f_s")
    ylabel!(L"H^\prime(f)")
    # Ausgabe der Filterkoeffizienten (erste Spalte = b, zweite Spalte = a)
    filtercoefficients = [coefb(filter)'; coefa(filter)']'
  end

Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Scope(Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :label), Any["N"], Dict{Symbol,Any}(:className => "interact ",:style => Dict{Any,Any}(:padding => "5px 10px 0px 10px")))], Dict{Symbol,Any}(:className => "interact-flex-row-left")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :input), Any[], Dict{Symbol,Any}(:max => 10,:min => 2,:attributes => Dict{Any,Any}(:type => "range",Symbol("data-bind") => "numericValue: index, valueUpdate: 'input', event: {change: function (){this.changes(this.changes()+1)}}","orient" => "horizontal"),:step => 1,:className => "slider slider is-fullwidth",:style => Dict{Any,Any}()))], Dict{Symbol,Any}(:className => "interact-flex-row-center")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :p), Any[], Dict{Symbol,Any}(:attributes => Dict("data-bind" => "text: formatted_val")))], Dict{Symbol,Any}(:className => "interact-flex-row-right"))], Dict{Symbol,Any}(:className => "interact-flex-row interact-widget")), Dict{String,Tuple{Observables.AbstractObservable,Union{Nothing, Bool}}}("changes" => (Observable{Int64} with 1 listeners. Value:
0, nothing),"index" => (Observable{Int64} with 2 listeners. Value:
6, nothing)), Set{String}(), nothing, Asset[Asset("js", "knockout", "/home/guido/.julia/packages/Knockout/IP1uR/src/../assets/knockout.js"), Asset("js", "knockout_punches", "/home/guido/.julia/packages/Knockout/IP1uR/src/../assets/knockout_punches.js"), Asset("js", nothing, "/home/guido/.julia/packages/InteractBase/sOe2Z/src/../assets/all.js"), Asset("css", nothing, "/home/guido/.julia/packages/InteractBase/sOe2Z/src/../assets/style.css"), Asset("css", nothing, "/home/guido/.julia/packages/Interact/SbgIk/src/../assets/bulma_confined.min.css")], Dict{Any,Any}("changes" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"changes\"]()) ? (this.valueFromJulia[\"changes\"]=true, this.model[\"changes\"](val)) : undefined})")],"index" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"index\"]()) ? (this.valueFromJulia[\"index\"]=true, this.model[\"index\"](val)) : undefined})")]), WebIO.ConnectionPool(Channel{Any}(sz_max:32,sz_curr:0), Set{AbstractConnection}(), Base.GenericCondition{Base.AlwaysLockedST}(Base.InvasiveLinkedList{Task}(Task (runnable) @0x00007efc58cea710, Task (runnable) @0x00007efc58cea710), Base.AlwaysLockedST(1))), WebIO.JSString[WebIO.JSString("function () {\n    var handler = (function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init: function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            });\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0)\n                     return;\n                stringified(str);\n            });\n            ko.applyBindingsToNode(\n                element,\n                {\n                    value: stringified,\n                    valueUpdate: allBindings.get('valueUpdate'),\n                },\n                context,\n            );\n        }\n    };\n    var json_data = {\"formatted_vals\":[\"2\",\"3\",\"4\",\"5\",\"6\",\"7\",\"8\",\"9\",\"10\"],\"changes\":WebIO.getval({\"name\":\"changes\",\"scope\":\"4525293616598707656\",\"id\":\"3981904163451177413\",\"type\":\"observable\"}),\"index\":WebIO.getval({\"name\":\"index\",\"scope\":\"45252